# Notebook for the Capstone project

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.96 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  38.27 MB/s
Fetching package metadata .............
Solving package specifications: 

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml.html as lh
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [7]:
#Load the data from table to a list
doc = lh.fromstring(source.content)
tr_elements = doc.xpath('//tr')

[len(T) for T in tr_elements[:10]]

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (name)
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        
        #Increment i for the next column
        i+=1
[len(C) for (title,C) in col]

Postcode
Borough
Neighbourhood



[289, 289, 289]

In [28]:
#Create a Dictionary and data frame 
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df1=df[df.Borough != 'Not assigned']

df1 = df1.replace('\n','', regex=True)
df1.head(10)
#df1.columns


,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B


In [29]:
#Group by Neighbourhood and combine neighbourhoods for same Borough
#separated by ","
df1=df1.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: "%s" % ', '.join(x)).reset_index(name='Neighbourhood')

df1.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df1.shape

(103, 3)

In [31]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
df3=pd.merge(df1,df2,how='left',left_on=['Postcode'],right_on=['Postal Code'])
del df3['Postal Code']
df3.head(5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
